In [13]:
import pickle
import numpy as np
d = pickle.load(open('/data/DF3D/pose_result__data_paper_180918_MDN_CsCh_Fly1_001_SG1_behData_images.pkl', 'rb'), encoding='latin')
d.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 'meta', 'points2d', 'points3d'])

In [14]:
cam_id = 2
R =  d[cam_id]['R']
tvec = d[cam_id]['tvec']
intr = d[cam_id]['intr']
points3d = d['points3d']
points2d = d['points2d'][cam_id]

In [15]:
points3d.shape, points2d.shape, intr

((899, 38, 3), (899, 38, 2), array([[1.60410e+04, 0.00000e+00, 2.40000e+02],
        [0.00000e+00, 1.59717e+04, 4.80000e+02],
        [0.00000e+00, 0.00000e+00, 1.00000e+00]]))

In [8]:
R, tvec

(array([[ 2.95891326e-01, -1.69940554e-02, -9.55070430e-01],
        [ 4.61626700e-03,  9.99855497e-01, -1.63607686e-02],
        [ 9.55210455e-01,  4.32149419e-04,  2.95927018e-01]]),
 array([1.89636405e-01, 4.06063127e-04, 9.25055730e+01]))

# take single 3d point *p3* and calculaete 2d projection p2 = M*(R*p3+t)

In [16]:
p = points3d[0,0] # take a single x,y,z example 
p_r = np.dot(R,p) + tvec
proj = np.dot(intr, p_r)
proj = proj / proj[-1]
proj = proj[:2]

In [10]:
print('proj', proj)
print('points2d', points2d[0,0])

proj [520.41789972 165.46772543]
points2d [517.5 172.5]


# do projection in batch, much faster

In [11]:
points3d_new_cs =  np.squeeze(np.matmul(R, points3d[:,:,:,np.newaxis])) + tvec[np.newaxis, np.newaxis]
proj = np.squeeze(np.matmul(intr, points3d_new_cs[:,:,:,np.newaxis]))

print(proj.shape)
proj = proj / proj[:, :, [2]]
proj = proj[:,:, :2]

(899, 38, 3)


In [12]:
np.mean(np.abs(points2d[:,:19] - proj[:,:19]))

2.8294925350443108